<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
# autoreload updated and newly installed packages
# without having to restart Jupyter kernel
%load_ext autoreload
%autoreload 2
# Some little interaction with matplotlib
%matplotlib inline
# Avoid using Jedi for faster autocomplete (tab)
%config Completer.use_jedi = False

# Significance of fatigue cycles depending on their frequency

In [2]:
# Backwards compatibility of the cluster_df function with python 3.8.X
from __future__ import annotations

# Standard imports
import ast
import copy
import datetime
import platform
import os
import sys
import time
from typing import List, Any
from collections import ChainMap, defaultdict

# Non-standard imports
from pandas.tseries.frequencies import to_offset
from pathlib import Path
try:
    from tqdm import tqdm
except ImportError:
    print("\33[33mInstall the \33[1m'tqdm'\33[22m module in your python"
          "environment using:\n\n\33[30m>>> "
          "\33[1m\33[34mpip \33[22m\33[30minstall \33[1m\33[32mtqdm\33[22m\n")
import matplotlib
import matplotlib.pyplot as plt
import numba as nb
import numpy as np
import pandas as pd
import plotly.graph_objs as go

# py-fatigue import
import py_fatigue as pf
import py_fatigue.testing as test
# Paths of this tutorial and of the data
TUTORIAL_PATH = os.path.dirname(os.getcwd())
DATA_PATH = os.path.join(Path.home(), "Documents", "Work", "data")
if not TUTORIAL_PATH in sys.path:
    sys.path.append(TUTORIAL_PATH)
np.set_printoptions(threshold=sys.maxsize)

In [3]:
print(f"Platform: {platform.platform()}")
print(f"Python version: {sys.version}")
print(f"py-fatigue version: {pf.styling.TermColors.CBOLD}{pf.__version__}")

print(f"{pf.styling.TermColors.CWHITEBG}\
{pf.styling.TermColors.CBOLD}\
{pf.styling.TermColors.CBLUE}")
      
print(f"DATA_PATH = {DATA_PATH}")
print(f"TUTORIAL_PATH = {TUTORIAL_PATH}")

print(f"{pf.styling.TermColors.CEND}")

Platform: Linux-5.19.0-26-generic-x86_64-with-glibc2.35
Python version: 3.8.15 (default, Dec 22 2022, 15:11:48) 
[GCC 12.2.0]
py-fatigue version: 1.0.11

DATA_PATH = /home/pd/Documents/Work/data
TUTORIAL_PATH = /home/pd/Python/Packages/Github/py-fatigue



In [4]:
plt.rcParams["figure.figsize"] = (7, 3.5)
plt.rcParams["font.family"] = ["Garamond", "Sans-Serif"]
plt.rcParams["font.size"] = 13
plt.rcParams["lines.markersize"] = 3
plt.rcParams["axes.grid"] = True
plt.rcParams["axes.grid.which"] = "both"
plt.rcParams['axes.axisbelow'] = True
plt.rcParams["grid.linestyle"] = "-"
plt.rcParams["grid.color"] = "#888888"
plt.rcParams["axes.spines.right"] = True
plt.rcParams["axes.spines.top"] = True
plt.rcParams["xtick.bottom"] = True
plt.rcParams["xtick.minor.bottom"] = True
plt.rcParams["xtick.top"] = True
plt.rcParams["xtick.minor.top"] = True
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["xtick.labelbottom"] = True
plt.rcParams["ytick.left"] = True
plt.rcParams["ytick.minor.left"] = True
plt.rcParams["ytick.right"] = True
plt.rcParams["ytick.minor.right"] = True
plt.rcParams["ytick.direction"] = "in"
plt.rcParams["ytick.labelleft"] = True
plt.rcParams["image.cmap"] = "Paired"
plt.rcParams['axes.prop_cycle'] = matplotlib.rcsetup.cycler(
    'color',
    ['0C5DA5', '00B945', 'FF9500', 'FF2C00', '845B97', '474747', '9E9E9E']
)

## 1. Data handling

### Import the data from csv file

The following operations are run:

1. Read .csv file containing cyce-counts in the form of py-fatigue sparse json
2. Set 'timestamp' as index
3. Convert literal columns containing the json cycle-counts to python dictionaries

In [5]:
start = time.time()
# 1
print(f"\33[36m1. Read \33[1m.csv\33[22m file\33[0m")
df = pd.read_csv(os.path.join(DATA_PATH, "CC_BB_C01_ALL_2018_TO_2021.csv"))
# 2
print(f"\33[36m2. Set \33[1m'timestamp'\33[22m as index\33[0m")
df = df.set_index("timestamp")
df.index = pd.to_datetime(df.index)
# 3
for col in df.columns:
    if col.startswith("CC_"):
        tqdm.pandas(desc=f"\33[36m3. Converting \33[1m{col[:3]}{col[-3:]}\33[0m") 
        df[col] = df[col].progress_apply(
            lambda x: ast.literal_eval(x) if not x!=x else x
        )
end = time.time()
print(f"\nElapsed time is {np.round(end-start, 0)}, s.")

1. Read .csv file
2. Set 'timestamp' as index


3. Converting CC_SG6: 100%|███████████| 157861/157861 [00:21<00:00, 7206.65it/s]


Elapsed time is 258.0, s.


<div class="alert alert-block alert-warning">
<b> ⚠️ Fallback:</b> In case we mess up with the main dataframe (<tt>df</tt>) used for the subsequent analyses, we keep <tt>df_</tt> intact as a fallback.
</div>


In [6]:
# df = copy.deepcopy(df_)

### From `dict` to `CycleCount`

The following part is still in the "managing the input data" workflow, as it converts the rainflow dictionaries to `pf.CycleCount` instances.

In [7]:
# 4
start = time.time()
tqdm.pandas(desc=f"\33[36m4. From \33[1mdict \33[22mto \33[1mpf.CycleCount\33[0m")
for col in df.columns:
    if col.startswith("CC_"):
        df[col] = df.progress_apply(
            lambda x: pf.CycleCount.from_rainflow(
                x[col], name=col[13:], timestamp=x.name
            ) if not x[col]!=x[col] else x[col], axis=1
        )
end = time.time()
print(f"\nElapsed time is {np.round(end-start, 0)}, s.")

4. From dict to pf.CycleCount: 100%|██| 157861/157861 [00:30<00:00, 5164.85it/s]



Elapsed time is 199.0, s.


### Aggregate the `CycleCounts` in `df` by a time window

The following part of the notebook makes extensive usage of the aggregation functionalities provided by [pandas.Dataframe.agg()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html). 

<div class="alert alert-block alert-info">
<b> ℹ️ Data aggregation:</b> an operation that involves collecting and summarizing data in a way that allows for statistical analysis and helps businesses achieve their goals. It is an important part of data warehousing because it enables decision-making based on large amounts of raw data. By aggregating data, it is possible to forecast future trends and use predictive modeling to make informed decisions. Additionally, proper data aggregation techniques can help to prevent performance issues.
</div>

*In this notebook, the fatigue data available are aggregated in time using the **`CycleCount.__add__()`** method.*

The following operations are performed sequentially:

1. Optionally, a subset of the dataframe can be taken for the analysis.
2. A set of aggregations in the form of [pandas date offset frequency strings](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-offset-aliases) has to be assigned. In this example we use: `'T'` (aggregate timestamps within every minute), `'D'` (day), `'W'` (week), `'M'` (month), `'Q'` (quarter), `'Y'` (year). In addition, we use also the `'all'` aggregation which simply aggregates the entire time window.
3. Checks are run in order to verify that the aggregations have been assigned in increasing time-range order. This requirement is important in the final part (bar plot).
4. The `pf.cycle_count.utils.aggregate_cc()` is run. The function returns:
    - The dataframe aggregated by the key assigned (e.g. `'W'`)
    - The sequence of residuals and their order of appearence. This datum is interesting to compare with the results of the bar plot.


In [8]:
n_days = 365 * 2
wf = copy.deepcopy(df).head(144 * n_days)  # Select the needed days (144 × n_days)

In [9]:
aggregations = ["T", "D", "W", "M", "Q", "Y"]

In [ ]:
dict_df_agg = dict()
hc = dict()
common_dt = pd.to_datetime("2000-01-01")
if "all" in aggregations:
    aggregations.pop("all")
time_ = [common_dt + to_offset(to) for to in aggregations]
sorted_aggregations = [x for _, x in sorted(zip(np.argsort(time_), aggregations))]
# sorted_aggregations.append("all")
for aggr in sorted_aggregations:
    dict_df_agg[aggr] = pf.cycle_count.utils.aggregate_cc(wf, aggr)
#     dict_df_agg[aggr], hc[aggr] = pf.cycle_count.utils.aggregate_cc(wf, aggr, True)

1. Running checks on df.
2. Building the aggregation dict.
3. Aggregate df by 'T'.
4. Retrieving LFFD on aggregated df.
Elapsed time for 'T' aggregation is 45.0, s.

1. Running checks on df.
2. Building the aggregation dict.
3. Aggregate df by 'D'.


Trying to add to a non CycleCount object instance after 20 April 2018, 18:00.


Trying to add to a non CycleCount object instance after 04 May 2018, 06:00.
Trying to add to a non CycleCount object instance after 04 May 2018, 00:00.


Trying to add to a non CycleCount object instance after 22 May 2018, 08:00.
Trying to add to a non CycleCount object instance after 22 May 2018, 00:00.


No residuals_sequence found on 25 May 2018, 13:10.
Trying to add to a non CycleCount object instance after 25 May 2018, 12:00.
No residuals_sequence found on 25 May 2018, 13:40.


...

Trying to add to a non CycleCount object instance after 01 September 2020, 07:00.


Trying to add to a non CycleCount object instance after 26 September 2020, 01:10.
Trying to add to a non CycleCount object instance after 26 September 2020, 20:20.


Trying to add to a non CycleCount object instance after 11 October 2020, 07:00.


Trying to add to a non CycleCount object instance after 30 November 2020, 23:00.


4. Retrieving LFFD on aggregated df. -> 100% completed...  


In [ ]:
# plt_prmtr = "CC_BB_C01_TP_SG_LAT019_DEG325_0_nr1"
# pf.cycle_count.utils.plot_aggregated_residuals(
#     hc,
#     plt_prmtr,
# )

## 2. Fatigue calculation

### SN curves

We want to define a set of SN curves that produce some realistic damage figures. To this aim, we take one of the sn curves from DNV-RP-C203 (specifically the D curve with free corrosion), and use that as baseline.

Given $\bar{m}$, $\ln\bar{a}$, and a number of cycles $\bar{N}$ where we want the SN curves to intersect (e.g. $1E7$), the intersection stress range is given by:

$$
    \ln{\bar{\sigma}} = \dfrac{\ln{\bar{a}} - \ln{\bar{N}}}{\bar{m}}\\
$$

From the point $\left(\ln{\bar{N}}, \ln{\bar{\sigma}} \right)$, it is possible to calculate the intersect $\ln{a}$ for each slope $m$ as:

$$
    \ln{a} = m \ln{\bar{\sigma}} + \ln{\bar{N}}
$$

In [ ]:
m_hat = 3
log_a_hat = 11.687
log_N_hat = 7
log_sigma_hat = (log_a_hat - log_N_hat) / m_hat

if log_sigma_hat <= 0:
    raise ValueError("log(a) <= log(N).\n\nThat is not possible.")

log_a_hat_m4 = 4 * log_sigma_hat + log_N_hat
log_a_hat_m5 = 5 * log_sigma_hat + log_N_hat

sn = defaultdict()
sn['DNV-D-C (m=3)'] = pf.SNCurve(
    slope=m_hat,
    intercept=log_a_hat,
    curve="D, m=3",
    environment="Free corrosion",
    norm="DNVGL-RP-C203",
    unit_string="MPa"
)

sn['DNV-D-C (m=4)'] = pf.SNCurve(
    slope=4,
    intercept=log_a_hat_m4,
    curve="D, m=4",
    environment="Free corrosion",
    norm="DNVGL-RP-C203",
    unit_string="MPa"
)

sn['DNV-D-C (m=5)'] = pf.SNCurve(
    slope=5,
    intercept=log_a_hat_m5,
    curve="D, m=5",
    environment="Free corrosion",
    norm="DNVGL-RP-C203",
    unit_string="MPa"
)


In [ ]:
fig, ax = plt.subplots()
for k, s in sn.items():
    s.plot(fig=fig, ax=ax)
plt.legend()
plt.show()

### Damage calculation through Palmgren-Miner rule

The Palmgren-Miner rule is applied to the aggregated damages using the `py_fatigue.cycle_count.utils.calc_aggregated_damage()` function which takes as input the aggregated dataframe and a dict or sequence of SN curves.

In [ ]:
dict_df_dmg = dict()
for aggr, df_d in dict_df_agg.items():
    dict_df_dmg[aggr] = pf.cycle_count.utils.calc_aggregated_damage(df_d, sn)

The variable `dict_df_dmg` is a dictionary that contains the date offset frequency strings as keys and the corresponding multi-indexed dataframes as values.

The multi-index dataframe contains the `'timestamp'` and `'sn_curve'` columns as indices, and the cycle-counted parameters names as column names.

To access a multi-index dataframe from within the `dict_df_dmg` dictionary, you can follow this logic:

```python
aggr_key = "W"
sn_curve_lookup = "m=[4.]"
dict_df_dmg[aggr_key].loc[sn_curve_lookup]
```

In [ ]:
dict_df_dmg["M"].loc["m=[3.]"]

### Aggregation of damages into total damage

The goal is obtaining one damage per date offset string aggregation (e.g. `'W'`), per parameter (e.g. `CC_...`), per SN curve. Therefore, the damages, grouped by date offset string and SN curve, are aggregated using the `sum()`.

As above, the resulting damages are stored in a multi-index dataframe which contains the `'aggregate_by'` and `'sn_curve'` columns as indices, the cycle-counted parameters names as column names, and the final damages in the cells.

To access this multi-index dataframe, follow for example this logic:

```python
sn_curve_lookup = "m=[4.]"
df_multiindex.loc[sn_curve_lookup]
```



In [ ]:
df_dmg_agg = pd.DataFrame()
for aggr, df_d in dict_df_dmg.items():
    
    df_1 = df_d.groupby(["sn_curve"]).agg("sum")
    df_1["aggregate_by"] = aggr
    df_dmg_agg = pd.concat([df_dmg_agg, df_1])
# df_dmg_agg = df_dmg_agg.transpose()
df_dmg_agg = df_dmg_agg.set_index([df_dmg_agg.index, "aggregate_by"])

### Damage normalization with respect to the total

The subdataframe (for fixed SN curve) containing the damages for each parameter and for each aggregation string is normalized with respect to the `'all'` row that containing the damages due to all the frequencies.

For this operation to give the correct results, the aggregation strings must be sorted in increasing time-range order. This operation has been performed in precedence.

- Show minor gridlines and show every 5-10% major
- Remove faulty sensors
- Specify that we also account for the residuals of the residuals as half cycles and that the "resolved residuals" result in both full and half cycles. This is the cause of the additional damage.

- For Marc: we were working exactly on this and to share the finished result. See for m=3 you have little influence of aggregations (coherent with the factor ~1.1), while m=5 there is a substantial effect of even monthly cycles. It is a site-dependent observation, but knowing the North Sea usual weather, this is a consistent result.

In [ ]:
fig, axes = plt.subplots(1, len(sn), figsize=(10, 3), dpi=600)

for j, sn_key in enumerate(["m=[3.]", "m=[4.]", "m=[5.]"]):
    
    df_dmg_end = df_dmg_agg.loc[sn_key]
    df_norm = df_dmg_end.div(df_dmg_end.max())
    df_norm = pd.concat([df_norm.loc["T"].to_frame().T, df_norm.diff()[1:]],
                         ignore_index=True) * 100
    df_norm.index = df_dmg_end.index
    df_norm.columns = [name.split('_')[1] for name in df_norm.columns]
    df_norm.transpose().plot.bar(stacked=True, ax=axes[j], legend=False, width=0.8)
    axes
    axes[j].set_title(f"m = {sn_key[3]}")
    axes[j].set_yticks(np.linspace(0, 100, 11, dtype=int))
    axes[j].set_xlabel("Parameter")
    axes[j].minorticks_on()
    axes[j].grid(visible=True, which="minor", color="#AAAAAA", linestyle=":")
    if j == 0:
        axes[j].set_ylabel("Normalized Damage, %")
    if j == 1:
        axes[j].legend(title="Aggregated by", loc="lower center", fancybox=True,
                   bbox_to_anchor=(0.5, 1.1), ncol=6, shadow=False)